<a href="https://colab.research.google.com/github/ddmitry80/otus-clickhouse/blob/main/OTUS_JOIN_agg_for_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка ClickHouse
# Во время загрузки потребует ввести ПАРОЛЬ!
# Если в конце будет Ok! значит установка завершена
!sudo apt-get update
!sudo apt-get install curl gnupg2 apt-transport-https ca-certificates
!sudo sh -c 'mkdir -p /usr/share/keyrings && curl -s https://builds.altinity.cloud/apt-repo/pubkey.gpg | gpg --dearmor > /usr/share/keyrings/altinity-archive-keyring.gpg'
!sudo sh -c 'echo "deb [signed-by=/usr/share/keyrings/altinity-archive-keyring.gpg] https://builds.altinity.cloud/apt-repo stable main" > /etc/apt/sources.list.d/altinity-dev.list'
!sudo apt-get update
#!sudo apt-get install clickhouse-common-static=21.8.10.1.altinitystable clickhouse-client=21.8.10.1.altinitystable clickhouse-server=21.8.10.1.altinitystable
!sudo apt-get install clickhouse-common-static=24.3.5.47.altinitystable clickhouse-client=24.3.5.47.altinitystable clickhouse-server=24.3.5.47.altinitystable
!sudo apt-get install clickhouse-client clickhouse-server

!sudo service clickhouse-server start
!sudo service clickhouse-server status
!curl localhost:8123

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,523 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,907 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all

In [ ]:
# Если хочется поработать с ClickHouse не через Colab
# Регаемся на https://ngrok.com/
# ngrok — это сервис, который позволяет открыть доступ к внутренним ресурсам машины,
# на которой он запущен, из внешней сети, путем создания публичного адреса,
# все запросы на который будут переброшены на локальный адрес и заданный порт.

!pip install pyngrok

# токен можно найти тут https://dashboard.ngrok.com/get-started/setup
!ngrok authtoken 2kNHm66ePlrYxnCzGLuUTBMCel6_7NCxMyaffUuW99134h1VY

# Эта команда отображет веб морду на другой адрес
# Адрес можно посмотреть тут https://dashboard.ngrok.com/cloud-edge/endpoints
!nohup ngrok http 8123 > /dev/null &

# Не забудьте в адресную строку дописать /play, чтобы видеть не просто "Ок.", а приложение, в которое можно писать запросы
# должно получится что-то вида https://eef0-34-74-98-209.ngrok-free.app/play

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
nohup: redirecting stderr to stdout


In [ ]:
# Работать будем через TCP-протокол, который реализован через библиотеку clickhouse-driver
!pip install clickhouse-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.0 MB/s eta 0:00:00


In [ ]:
from clickhouse_driver import Client

client = Client(host='localhost', user='default', password='12345')

# JOINs

## Основные типы соединений

In [ ]:
# Создаем и заполняем таблицы

client.execute('''
CREATE TABLE table1
(
    id UInt64,
    date DateTime,
    value UInt64
)
ENGINE = Log
''')

client.execute('''
INSERT INTO table1 (*) VALUES (0, '2020-01-01 10:06:00', 50),
(1, '2020-01-01 12:01:07', 12),
''')

client.execute('''
CREATE TABLE table2
(
    id UInt64,
    date DateTime,
    description String
)
ENGINE = Log
''')

client.execute('''
INSERT INTO table2 (*) VALUES (0, '2020-01-01 10:00:05', 'cancel'),
(0, '2020-01-01 10:05:05', 'purchase'),
(3, '2020-01-01 12:01:07', 'purchase')
''')

[]

In [ ]:
client.execute('''SELECT * FROM table1 JOIN table2 ON table1.id = table2.id''')

[(0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  0,
  datetime.datetime(2020, 1, 1, 10, 0, 5),
  'cancel'),
 (0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  0,
  datetime.datetime(2020, 1, 1, 10, 5, 5),
  'purchase')]

In [ ]:
client.execute('''SELECT * FROM table1 left JOIN table2 using(id)''')

[(0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  datetime.datetime(2020, 1, 1, 10, 0, 5),
  'cancel'),
 (0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  datetime.datetime(2020, 1, 1, 10, 5, 5),
  'purchase'),
 (1,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  12,
  datetime.datetime(1970, 1, 1, 0, 0),
  '')]

In [ ]:
# Создаем таблицу, где существуют столбцы с NULL

client.execute('''
CREATE TABLE table3
(
    id UInt64 NULL,
    date DateTime NULL,
    description String NULL
)
ENGINE = Log()
''')

client.execute('''
INSERT INTO table3 (*) VALUES (0, '2020-01-01 10:00:05', 'cancel'),
(0, '2020-01-01 10:05:05', 'purchase'),
(3, '2020-01-01 12:01:07', NULL)
''')

[]

In [ ]:
client.execute('''SELECT * FROM table1 left JOIN table3 using(id)''')

[(0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  datetime.datetime(2020, 1, 1, 10, 0, 5),
  'cancel'),
 (0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  datetime.datetime(2020, 1, 1, 10, 5, 5),
  'purchase'),
 (1, datetime.datetime(2020, 1, 1, 12, 1, 7), 12, None, None)]

In [ ]:
client.execute('''SELECT * FROM table1 full JOIN table2 using(id)''')

[(0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  datetime.datetime(2020, 1, 1, 10, 0, 5),
  'cancel'),
 (0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  datetime.datetime(2020, 1, 1, 10, 5, 5),
  'purchase'),
 (1,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  12,
  datetime.datetime(1970, 1, 1, 0, 0),
  ''),
 (3,
  datetime.datetime(1970, 1, 1, 0, 0),
  0,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  'purchase')]

In [ ]:
client.execute('''SELECT * FROM table1 cross JOIN table3''')

[(0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  0,
  datetime.datetime(2020, 1, 1, 10, 0, 5),
  'cancel'),
 (0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  0,
  datetime.datetime(2020, 1, 1, 10, 5, 5),
  'purchase'),
 (0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  3,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  'purchase'),
 (1,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  12,
  0,
  datetime.datetime(2020, 1, 1, 10, 0, 5),
  'cancel'),
 (1,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  12,
  0,
  datetime.datetime(2020, 1, 1, 10, 5, 5),
  'purchase'),
 (1,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  12,
  3,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  'purchase')]

## Дополнительные типы соединений

In [ ]:
client.execute('''SELECT * FROM table1 LEFT SEMI JOIN table2 using(id) ''')

[(0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  datetime.datetime(2020, 1, 1, 10, 0, 5),
  'cancel')]

In [ ]:
client.execute('''SELECT * FROM table1 LEFT anti JOIN table2 using(id) ''')

[(1,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  12,
  datetime.datetime(1970, 1, 1, 0, 0),
  '')]

In [ ]:
client.execute('''SELECT * FROM table1 LEFT any JOIN table2 using(id) ''')

[(0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  datetime.datetime(2020, 1, 1, 10, 0, 5),
  'cancel'),
 (1,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  12,
  datetime.datetime(1970, 1, 1, 0, 0),
  '')]

In [ ]:
client.execute('''SELECT * FROM table1 ASOF JOIN table2 using(id, date)''')

[(0, datetime.datetime(2020, 1, 1, 10, 6), 50, 'purchase')]

In [ ]:
client.execute('''SELECT * FROM table1 paste JOIN table2''')

[(0,
  datetime.datetime(2020, 1, 1, 10, 6),
  50,
  0,
  datetime.datetime(2020, 1, 1, 10, 0, 5),
  'cancel'),
 (1,
  datetime.datetime(2020, 1, 1, 12, 1, 7),
  12,
  0,
  datetime.datetime(2020, 1, 1, 10, 5, 5),
  'purchase')]

# Проблемы JOINs в Clickhouse и их решения

In [ ]:
# Создаем таблицы с 2 млн. и 2 млрд. строк

client.execute('''CREATE TABLE 2billion (idx Int64) ENGINE = Log''')
client.execute('''CREATE TABLE 2million (idx Int64) ENGINE = Log''')

client.execute('''INSERT INTO 2billion (idx) select * from numbers(1, 2000000000)''')
client.execute('''INSERT INTO 2million (idx) select * from numbers(1, 2000000000, 1000)''')


In [ ]:
client.execute('''
SET send_logs_level='trace';
select count(*)
from 2billion
left join 2million using(idx)
''')

In [ ]:
client.execute('''
SET send_logs_level='trace';
select count(*)
from 2million
join 2billion using(idx)
''')

In [ ]:
client.execute('''
SET send_logs_level='trace';
select count(*)
from 2billion
right join 2million using(idx)
''')

In [ ]:
client.execute('''
SET send_logs_level='trace';
select count(*)
from 2million as 2m
join (select *
from 2billion
where idx IN (select idx
from 2million)) as 2m
using(idx)
''')

In [ ]:
client.execute('''
SET send_logs_level='trace';
select count(*)
from 2million
join 2billion using(idx)
SETTINGS join_algorithm = 'full_sorting_merge'
''')

# Агрегатные функции

In [ ]:
#-If
client.execute('''
SELECT countIf(toHour(date)>10) AS after_10 FROM table2;
''')

In [ ]:
#-Distinct
client.execute('''
SELECT countDistinct(description) AS uniq_events FROM table2;
''')

In [ ]:
# -ForEach
client.execute('''
CREATE TABLE table_arr
(
    id UInt64,
    arr Array(UInt32)
)
ENGINE = Log
''')

client.execute('''
INSERT INTO table_arr (*) VALUES (1, [1, 2, 3]),
(1, [2, 3, 7]),
(1, [9, 1, 1])
''')

client.execute('''
SELECT maxForEach(arr) FROM table_arr
''')

In [ ]:
# - State
# - Merge
# -MergeState

In [ ]:
# -Resample
client.execute('''
SELECT groupArrayResample(0, 4, 2)(description, id) FROM table2;
''')

In [ ]:
# NULL
client.execute('''SELECT isNull(description) FROM table3''')

# NaN
client.execute('''SELECT NaN, 0/0, toTypeName(NaN)''')

# Inf
client.execute('''select Inf, -Inf, 1/, toTypeName(Inf)''')


In [ ]:
client.execute('''SELECT groupArray(description) FROM table3 ''')

# count()=count(*), count() - чисто кликхаусовский вариант написания
client.execute('''SELECT count() FROM table 3''')

client.execute('''SELECT count(description) FROM table 3 ''')